In [1]:
# first, let us create our inputs
import gym 
from nn import PolicyNN, ValueNN

# lets get our environment
env = gym.make("CartPole-v1")

# Policy parameters (NN) = "theta". 2 hidden layers w/ 32 nodes. softmax output for each action
theta = PolicyNN(
    env.observation_space.shape[0],
    2,
    32,
    env.action_space.n
    )

# Value function parameters (NN) = "V". outputs value of state (scalar)
V = ValueNN(
    env.observation_space.shape[0],
    2,
    32,
    1
)

In [ ]:
# step 1, generate trajectories
'''
As input, we will need:
    - the gym environment
    - a policy "pi" which takes in the state and outputs an action
As output, we return:
    - the trajectory of (s, a, r) tuples
'''
def generate_trajectory(env, pi, render_last_step: bool=False ):
    episode = []
    s = env.reset()
    a = pi(s)
    done = False
    
    while not done:
        s_p, r, done, info = env.step(a)
        episode.append((s, a, r))
        s = s_p
        a = pi(s_p)
        if done:
            env.render()

    return episode

In [ ]:
# step 2, estimate the advantage of the trajectory
'''
Note: Advantage = A(a, s) = sum(r_t + gamma^(t+1) * V(s_t+1) - V(s_t)) where t = 0,1,2,...,T-1, and T = len(trajectory)
As input, we will need:
    - the trajectory, list of (s, a, r) tuples
    - gamma (discount)
    - value function V which takes in the state and outputs a scalar value
As output, we return:
    - a list of scalars which is the discounted advantage at every step 
'''
from typing import List
def advantage_estimates(trajectory, gamma : float, V) -> List[float]:
    advantages = {}
    for t in range(0, len(trajectory)-1):
        s_t, a_t, r_t = trajectory[t]
        s_t_1, _, _ = trajectory[t+1]
        advantages[(s_t, a_t)] = (r_t + gamma * V(s_t_1) - V(s_t))
    return advantages

In [ ]:
# step 3, sample estimates for objective L_theta_k and KL-Divergence constraint H using advantage estimates
'''
Single path sample estimates for L_theta_k and H
Equations:
    - g_k = gradient_theta(L_theta_k(theta))
    - L_theta_k = sum(discounted future rewards -> just use advantage), where s_0~p_0, a~q, q(a|s) = policy_theta_k(a|s)
    - H = FIM = sum(KL_divergence(pi_theta_old(*|s_n) || pi_theta(*|s_n)))
Inputs:
    - policy_theta_k
    - DONT need action distribution (for single path, this will be equivalent to our policy)
    - number of samples
    - to pass along
        - env (calling generate trajectory)
        - V (calling advantage estimate)
Outputs:
    - estimated objective L_theta_k(theta)
    - estimated constraint KL divergence H 
'''

def single_path_sample_estimator(pi : PolicyNN, num_samples: int):
    # set of trajectories D_k
    D_k = []
    for i in range(num_samples):
        D_k.append(generate_trajectory(env, p))
    return